In [2]:
def load_annotation(path):
    ann = defaultdict(list)
    with open(path, "r") as f:
        for line in f:
            line = line.rstrip()
            if not line:
                continue
            line = json.loads(line)
            line["page_id"] = int(line["page_id"])
            ann[line["page_id"]].append(line)
    return ann

In [18]:
from pathlib import Path
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
base_path = Path("/data1/ujiie/shinra/tohoku_bert/")

import json

for path in base_path.glob("**/"):
    if path.stem in ["tohoku_bert", "_temp_files", "tokens"]:
        continue
    if path.parent.stem == "tohoku_bert":
        continue

    if not (path / "tokens").exists():
        continue

    category = path.stem

    diff_cnt = 0
    total = 0

    anns = load_annotation(path / f"{category}_dist.json")
    ids = np.array(list(anns.keys()))
    train_ids, test_ids = train_test_split(ids, test_size=0.2)
    train_ids, valid_ids = train_test_split(train_ids, test_size=0.25)

    (base_path / "data_split" / category).mkdir(exist_ok=True)
    with open(base_path / "data_split" / category / "train.txt", "w") as f:
        f.write("\n".join(train_ids.astype(str)))
    with open(base_path / "data_split" / category / "valid.txt", "w") as f:
        f.write("\n".join(valid_ids.astype(str)))
    with open(base_path / "data_split" / category / "test.txt", "w") as f:
        f.write("\n".join(test_ids.astype(str)))

    